# Sequence to Sequence (a.k.a. seq2seq)

**학습목표**
* Encoder Decoder 구조를 이해하고 구현할 줄 안다.
* Seq2Seq에 필요한 전처리를 이해한다.

![이런거](https://raw.githubusercontent.com/KerasKorea/KEKOxTutorial/master/media/28_1.png)
---------------------------------
edu.rayleigh@gmail.com
Special Thanks to : 숙번님 ( [봉수골 개발자 이선비](https://www.youtube.com/channel/UCOAyyrvi7tnCAz7RhH98QCQ) )

In [11]:
# !wget http://www.manythings.org/anki/fra-eng.zip

In [12]:
# import zipfile
# fra_eng = zipfile.ZipFile('fra-eng.zip')
# fra_eng.extractall()
# fra_eng.close()

In [13]:
!wget https://raw.githubusercontent.com/L1aoXingyu/seq2seq-translation/master/data/eng-fra.txt

--2023-03-31 06:41:49--  https://raw.githubusercontent.com/L1aoXingyu/seq2seq-translation/master/data/eng-fra.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9541158 (9.1M) [text/plain]
Saving to: ‘eng-fra.txt.1’

eng-fra.txt.1       100%[===================>]   9.10M  --.-KB/s    in 0.1s    

2023-03-31 06:41:50 (91.8 MB/s) - ‘eng-fra.txt.1’ saved [9541158/9541158]



In [14]:
import pandas as pd
# temp = pd.read_table('fra.txt', names=['Eng', 'Fra', 'License'])
temp = pd.read_table('eng-fra.txt', names=['Eng', 'Fra'])
temp.shape

(135842, 2)

In [15]:
temp.head()

,Eng,Fra
0,Go.,Va !
1,Run!,Cours !
2,Run!,Courez !
3,Wow!,Ça alors !
4,Fire!,Au feu !


# 너무 많으므로 50000개 문장만 진행하자.

In [16]:
# ## 끔찍한 결과를 볼 수 있다.
# temp = temp.sample(n=50000, replace=False, random_state=2021)

temp = temp.iloc[:50000]

In [17]:
eng_sent = temp['Eng'].tolist()
fra_sent = temp['Fra'].tolist()

In [18]:
print(eng_sent[100])
print(fra_sent[100])

Go away!
Pars !


# 데이터 준비
0. 단어와 구두점 사이 공백 만들기
1. sos 와 eos
1. tokenizing, idx_seq, padding

## 0. 단어와 구두점 사이 공백 만들기


In [19]:
ex1 = 'Abandonne\u202f!'
ex2 = 'Allez vous échauffer !'

In [20]:
# 정규식 공부
# 문장을 하나 딱 짚어서
# 실제 어떻게 동작하는지를 뜯어서 확인

import unicodedata
import re
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,'¿])", r" \1 ", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?']+", r" ", sent)
    sent = re.sub(r"\s+", " ", sent)
    return sent

In [21]:
preprocess_sentence("I'm just a poor boy.")

"i ' m just a poor boy . "

In [22]:
eng_sent = [ preprocess_sentence(sent) for sent in eng_sent ]
fra_sent = [ preprocess_sentence(sent) for sent in fra_sent ]

In [23]:
print(eng_sent[100])
print(fra_sent[100])

go away ! 
pars ! 


## 1. sos 와 eos
1. sos : start of speech
2. eos : end of speech

In [24]:
fra_sent = [f"<sos> {fra} <eos>" for fra in fra_sent]
fra_sent[100]

'<sos> pars !  <eos>'

## 2. Tokenizing, idx_seq, padding

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [26]:
# Tokenizing
tokenizer_en = Tokenizer(filters="", lower=True)
tokenizer_en.fit_on_texts(eng_sent)
tokenizer_fr = Tokenizer(filters="", lower=True)
tokenizer_fr.fit_on_texts(fra_sent)

In [27]:
# Index Sequence
eng_seq = tokenizer_en.texts_to_sequences(eng_sent)
fra_seq = tokenizer_fr.texts_to_sequences(fra_sent)

print(eng_seq[100])
print(fra_seq[100])

[38, 194, 47]
[1, 683, 18, 2]


In [36]:
# padding   프랑스어 패딩 위치를 '뒤'로 바꿨음.
eng_pad = pad_sequences(eng_seq, padding='pre') # 최대 문장 길이에 패딩에 맞춰지게 됨.
fra_pad = pad_sequences(fra_seq, padding='post')

print(eng_pad[100])
print(fra_pad[100])
print(eng_pad.shape)
print(fra_pad.shape)

[  0   0   0   0   0   0   0   0  38 194  47]
[  1 683  18   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]
(50000, 11)
(50000, 19)


In [29]:
# tokenizer에서 0 index가 구성되어있지 않지만, 
# pad_sequence에서 pad의 의미로 0을 사용하고 있어서, 전체 사이즈를 구할 때, +1을 해준다.

eng_vocab_size = len(tokenizer_en.word_index) + 1
fra_vocab_size = len(tokenizer_fr.word_index) + 1
print("영어 단어 집합의 크기: {:d}\n프랑스어 단어 집합의 크기: {:d}".format(eng_vocab_size, fra_vocab_size))

영어 단어 집합의 크기: 5965
프랑스어 단어 집합의 크기: 10406


# 모델링!

In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, GRU

In [31]:
# 혹시 이미 그려둔 그래프가 있다면 날려줘!
tf.keras.backend.clear_session()

# 영어 단어 집합의 크기 : 5965, (50000, 11)
# 프랑스어 단어 집합의 크기 : 10406, (50000, 19)
# 프랑스어 문장은 길이가 19이지만,
# 디코더의 인풋으로 넣을때는 맨 뒤의 <eos>를 떼고 길이 18의 문장을
# 디코더의 아웃풋은 맨 앞의 <eos>를 떼고 길이 18의 문장으로 준비해야 함.

# Encoder
enc_X = tf.keras.layers.Input(shape=[eng_pad.shape[1]])
enc_E = tf.keras.layers.Embedding(eng_vocab_size, 64)(enc_X) # 토큰수, 차원수
enc_S_full, enc_S = tf.keras.layers.GRU(256, return_sequences=True, return_state=True)(enc_E)
## 물론 지금은 enc_S_full은 사용하지 않는다.
# S : (Hidden) State

# Decoder
dec_X = tf.keras.layers.Input(shape=[fra_pad.shape[1]-1])
dec_E = tf.keras.layers.Embedding(fra_vocab_size, 64)(dec_X) # 토큰수, 차원수
dec_H = tf.keras.layers.GRU(256, return_sequences=True)(dec_E, initial_state=enc_S)
# dec_H = tf.keras.layers.Dense(256, activation="swish")(dec_H) # 없어도 상관은 없는 부분.
dec_Y = tf.keras.layers.Dense(fra_vocab_size, activation="softmax")(dec_H) # 매시점에서, 어떤 단어가 타당할지 분류 문제로 푸는 것

model = tf.keras.models.Model([enc_X, dec_X], dec_Y)
# 텍스트는 index이고(원핫인코딩을 안했고)
# 아웃풋레이어는 분류문제 처럼 노드가 준비되어 있다면
# sparse categorical crossentropy
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 11)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 18)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 11, 64)       381760      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 18, 64)       665984      ['input_2[0][0]']                
                                                                                              

In [37]:
# decoder의 인풋은 마지막 <eos>를 뗀다.
# decoder의 아웃풋 학습시엔 처음의 <sos>를 뗀다.
model.fit([eng_pad, fra_pad[:, :-1]], fra_pad[:, 1:], shuffle=True, 
          batch_size=128, epochs=10)

Epoch 1/10
391/391 [==============================] - 13s 34ms/step - loss: 1.1938 - accuracy: 0.7906
Epoch 2/10
391/391 [==============================] - 13s 34ms/step - loss: 1.0997 - accuracy: 0.8055
Epoch 3/10
391/391 [==============================] - 13s 34ms/step - loss: 1.0576 - accuracy: 0.8108
Epoch 4/10
391/391 [==============================] - 13s 34ms/step - loss: 1.0238 - accuracy: 0.8150
Epoch 5/10
391/391 [==============================] - 13s 34ms/step - loss: 0.9939 - accuracy: 0.8187
Epoch 6/10
391/391 [==============================] - 14s 35ms/step - loss: 0.9667 - accuracy: 0.8224
Epoch 7/10
391/391 [==============================] - 13s 34ms/step - loss: 0.9411 - accuracy: 0.8252
Epoch 8/10
391/391 [==============================] - 13s 34ms/step - loss: 0.9167 - accuracy: 0.8285
Epoch 9/10
391/391 [==============================] - 13s 33ms/step - loss: 0.8933 - accuracy: 0.8315
Epoch 10/10
391/391 [==============================] - 13s 34ms/step - loss: 0.870

In [40]:
import numpy as np

# 영어 단어 집합의 크기 : 5965, (50000, 11)
# 프랑스어 단어 집합의 크기 : 10406, (50000, 19)

def translate(eng):  # 영어 문장을 인풋으로 받아서
    # eng => index => pad
    eng_seq = tokenizer_en.texts_to_sequences([eng]) # 인덱스의 시퀀스로 바꾸고
    eng_pad = tf.keras.preprocessing.sequence.pad_sequences(eng_seq, maxlen=11) # 문장길이 통일해줍니다.

    fra = []  # 번역: 시점순서에 맞추어 차근차근 단어를 선택하여 담을 공간
    for n in range(19-1):
        # fra => index => pad
        # 첫 루프에는 <sos>만 담겨있습니다.
        fra_seq = tokenizer_fr.texts_to_sequences([['<sos>'] + fra]) # 인덱스의 시퀀스로 바꿉니다.
        # 문장길이 통일합니다. 문장 앞부분에 0으로 가득차게 됩니다. 
        # 공부한 전략과 다르지요? 원래는 LSTM 시점 하나하나 조절해야 하는데
        # 코드 난이도를 낮추기 위해 변형을 가했습니다.
        fra_pad = tf.keras.preprocessing.sequence.pad_sequences(fra_seq, maxlen=19-1) 
        # 영어문장과, 현재까지 번역한 프랑스어를 input(번역한게 없으면 <sos>만)으로 사용합니다.
        # eng_pad는 인코더 인풋으로,  fra_pad는 디코더 인풋으로 갑니다.
        fra_next = model.predict([eng_pad, fra_pad]) 

        # onehot -> index -> word
        fra = [tokenizer_fr.index_word[i] for i in np.argmax(fra_next[0], axis=1) if i != 0]
        # fra_next는 모든 단어 클래스 별 확률값이 담겨 있고
        # 그 중 가장 확률이 높은 단어의 인덱스를 선택하여
        # 토큰을 복원하는 과정입니다.
        # 그래서 fra 리스트에 선택된 token을 하나하나 담습니다.
        # 0번째를 무시하는 이유는 <sos>가 필요없어서!

        # 번역된 word 선택
        fra = fra[:n+1]
        print(fra)  # 무슨 과정이 일어나는지 눈으로 추적 가능
        
        if fra[-1] == '<eos>':  # 마지막이 eos면 본 과정을 마무리 합니다.
            break

    return fra

# 위 함수를 우리가 배운 전략 그대로 사용하려면
# https://tykimos.github.io/2018/09/14/ten-minute_introduction_to_sequence-to-sequence_learning_in_Keras/
# 위 링크를 참고하면 좋습니다.
# 단... 코드 난이도가 좀 올라갑니다.

In [41]:
translate("I am a boy.")  # 무슨일이 일어나는지 추적 용도

1/1 [==============================] - 0s 27ms/step
['je']
1/1 [==============================] - 0s 19ms/step
['je', 'suis']
1/1 [==============================] - 0s 23ms/step
['je', 'suis', '!']
1/1 [==============================] - 0s 21ms/step
['je', 'suis', '!', '<eos>']


['je', 'suis', '!', '<eos>']

In [39]:
import random

# 랜덤 10개
indices = list(range(50000))
random.shuffle(indices)

for n in indices[:10]:
    print(f"영어: {eng_sent[n]}\n불어: {fra_sent[n]}")
    print(f"번역: {' '.join(translate(eng_sent[n])[:-1])}")
    print()

영어: i nearly starved . 
불어: <sos> je mourus presque de faim .  <eos>
1/1 [==============================] - 0s 21ms/step
['je']
1/1 [==============================] - 0s 20ms/step
['je', 'suis']
1/1 [==============================] - 0s 21ms/step
['je', 'suis', '!']
1/1 [==============================] - 0s 21ms/step
['je', 'suis', '!', '<eos>']
번역: je suis !

영어: i was too small . 
불어: <sos> j ' etais trop petit .  <eos>
1/1 [==============================] - 0s 19ms/step
['j']
1/1 [==============================] - 0s 22ms/step
['j', 'de']
1/1 [==============================] - 0s 19ms/step
['j', 'de', "'"]
1/1 [==============================] - 0s 19ms/step
['j', 'de', "'", 'ete']
1/1 [==============================] - 0s 18ms/step
['j', 'de', "'", 'ete', "'"]
1/1 [==============================] - 0s 19ms/step
['j', 'de', "'", 'ete', "'", '!']
1/1 [==============================] - 0s 20ms/step
['j', 'de', "'", 'ete', "'", '!', "'"]
1/1 [==============================] - 0s 19ms/st